## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 2: AI Services

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os
from IPython.display import Image, display

from semantic_kernel import Kernel
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

In [2]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

### 🪜 Step 2: Add AI service

In [3]:
# Initialise kernel
kernel = Kernel()

In [4]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

### 🪜 Step 3: Retrieve AI service

In [5]:
# Retrieve the chat completion service by type
chat_completion_service = kernel.get_service(type=ChatCompletionClientBase)
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

In [6]:
# Retrieve the chat completion service by id
chat_completion_service = kernel.get_service(service_id="azure_openai_chat")
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

### 🪜 Step 4: Direct use of Chat Completion services

You can use the chat completion service directly, without adding to the kernel.

In [7]:
# Instantiatte execution settings
execution_settings = OpenAIChatPromptExecutionSettings()

### a) Non-streaming chat completion

In [8]:
# Initialise new chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [9]:
# Get non-streaming response
response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

print(response)

Deep within the misty forests of the Eastern Himalayas, a little red panda named Tashi lived among the towering bamboo stalks. Covered in a soft, reddish-brown fur and sporting a bushy ringed tail, Tashi spent her days climbing trees with remarkable agility, seeking out fresh bamboo shoots and juicy berries. Her keen eyes and nimble paws helped her navigate the canopy, where she found safety from predators and a peaceful place to rest.

One crisp autumn morning, Tashi noticed a flicker of movement near the forest floor. Curious, she carefully descended from her perch and discovered a tiny injured bird tangled in a patch of thorny bushes. Despite her small size, Tashi gently used her sharp claws to free the bird, then gathered leaves and moss to create a cozy nest for it to recover. Day by day, she stayed close by, sharing fresh berries and keeping watch until the bird’s wing healed.

As the seasons changed, Tashi and the grateful bird formed an unlikely friendship, exploring the forest

### b) Streaming chat completion

In [10]:
# Initialise new chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [11]:
# Get streaming response
response = chat_completion_service.get_streaming_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

async for chunk in response:
    print(chunk, end="")

In the misty forests of the Eastern Himalayas, a small creature with fiery red fur and a bushy ringed tail quietly went about its day. This was Rani, a curious red panda who loved to explore the towering bamboo groves and moss-covered trees that made up her home. Though she was about the size of a house cat, Rani was an excellent climber, deftly navigating the branches high above the forest floor to find tender bamboo shoots and juicy berries. Her gentle nature and shy demeanor kept her hidden from most predators, allowing her to live peacefully in the dappled sunlight filtering through the canopy.

Each morning, Rani would nibble on a breakfast of bamboo leaves while watching the birds flit past. Sometimes, she’d encounter other red pandas like herself, and they would exchange soft chuffing calls, a language unique to their kind. These encounters were brief yet meaningful, helping to strengthen bonds and establish territories without conflict. Despite their solitary tendencies, red pa

### 🪜 Step 5: Use rich messages in chat history

In [12]:
# Public image URL for demonstration
hotel_room_image_url = "https://images.pexels.com/photos/164595/pexels-photo-164595.jpeg"

print(f"Anya Sharma's reference image for a hotel room:")
display(Image(url=hotel_room_image_url, width=400)) # Display the image directly in the notebook

Anya Sharma's reference image for a hotel room:


In [13]:
# Initialise new chat history
chat_history = ChatHistory()

chat_history.add_message(
    ChatMessageContent(
        role = AuthorRole.SYSTEM,
        content = "You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible."
    )
)

chat_history.add_message(
    ChatMessageContent(
        role = AuthorRole.USER,
        name = "Anya_Sharma",
        items = [
            TextContent(text="I'm looking to book a room. Can you help me find something similar to what's in this picture?"),
            ImageContent(uri=hotel_room_image_url) # Referencing the image displayed previously
        ]
    )
)

In [14]:
# Simulate function call and result (Tool Message) for user's preferences
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.ASSISTANT,
        items=[
            FunctionCallContent(
                name = "get_user_preferences_HotelBookingPlugin",
                id = "user_prefs_anya_001",
                arguments = str({"username": "Anya_Sharma"})
            )
        ]
    )
)

# Tool provides the result of the function call (simulated)
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.TOOL,
        items=[
            FunctionResultContent(
                name = "get_user_preferences_HotelBookingPlugin",
                id = "user_prefs_anya_001",
                result = '{ "loyalty_status": "Diamond", "preferred_amenities": ["king_bed", "balcony", "high_floor", "late_checkout"] }'
            )
        ]
    )
)

In [15]:
# Get assistant's response
assistant_response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

chat_history.add_message(assistant_response)
print(f"Assistant: {assistant_response.content}")

Assistant: I see you prefer a room with a king bed, balcony, high floor, and late checkout. The room in the picture you provided has a modern and cozy design with a dark, sleek headboard and some art above the bed. I'll help you find a room that matches this style and your preferences.

Could you please let me know your preferred location and dates for the stay? This will help me find the best suitable options for you.


In [16]:
# Display the full chat history
print("\n--- Full Rich Chat History ---")
for message in chat_history:
    print(
        f"Role: {message.role}",
        f"Name: {message.name}",
        f"Content: {message.content}",
        f"Items: {message.items}\n"
    )


--- Full Rich Chat History ---
Role: AuthorRole.SYSTEM Name: None Content: You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible. Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='You are a helpful hotel booking assistant. You can assist users in finding and booking rooms, answer questions about hotel amenities, and consider their preferences. Provide personalised suggestions where possible.', encoding=None)]

Role: AuthorRole.USER Name: Anya_Sharma Content: I'm looking to book a room. Can you help me find something similar to what's in this picture? Items: [TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text="I'm looking to book a room. Can you help me find something similar to what's in this picture?", encoding=None), ImageContent(inner_conte